## Imports

In [1]:
import os
import math
import obspy
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from datetime import timedelta
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

from earthquake import earthquake
import util
import pandas as pd

## Get set up
Set path to data, and read eq_list (all folders in root folder)

In [2]:
root = '/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3/'

eq_list = os.listdir(root)

Open catalog of events

In [3]:
client = Client("IRIS")
# cat = client.get_events(starttime=UTCDateTime("2019-01-01"), endtime=UTCDateTime("2020-01-01"), minmagnitude=5, includearrivals=True)
cat = obspy.read_events('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3_catalog.xml')

Not all events had suitable data, look through all events and make a list of ones which have data (eq_with_data) and were successfully picked

In [4]:
eq_with_data = []
cat_with_data = cat.copy()
cat_with_data.clear()
for event in cat:
    eq_name = util.catEventToFileName(event)
    if os.path.isdir(root+eq_name) and os.path.isdir(root+eq_name+'/station_xml_files') and os.path.exists(root+eq_name+'/picks.pkl'):
        eq_with_data.append(eq_name)
        cat_with_data.extend([event])

## the action!!
For all events with data, load the data and then create an earthquake object. work out tp_max and tc, and add to lists (for plotting)

In [5]:
list_tpmax = []
list_mags = []
list_tc= []
list_eq = []
eqs = {}
count = 0
max_radius_km = 100
for eq_name in eq_with_data[0:3000]:
    
    d = util.filenameToDate(eq_name)
    
    filter_start = str(UTCDateTime(d-timedelta(seconds=1)))
    filter_stop = str(UTCDateTime(d+timedelta(seconds=1)))
    
    event = cat.filter('time > ' + filter_start, 'time < ' + filter_stop)
    # print(event)
    data = obspy.read(root+eq_name+'/data/*/*')
    inv = obspy.read_inventory(root+eq_name+'/station_xml_files/*')
    try:
        data.remove_response(inv)
    except:
        eq_with_data.remove(eq_name)
        count += 1
        continue
    with open(root+eq_name+'/picks.pkl', 'rb') as f:
        picks = pickle.load(f)
    obj_name = eq_name[0:-2]
    data_nearby = util.find_nearby_data(data, inv, event[0], max_radius_km)
    eqs[obj_name] = earthquake(eq_name, event, data_nearby, picks, sensor_types = [])
    eqs[obj_name].calc_Tpmax()
    eqs[obj_name].calc_Tc()
    list_tpmax.append(eqs[obj_name]._cached_params["tau_p_max"])
    list_tc.append(eqs[obj_name]._cached_params["tau_c"])
    list_mags.append(eqs[obj_name].event.magnitudes[0].mag)
    list_eq.append(eq_name)
    print('earthquake number ' + str(count) + ' of '+ str(len(eq_with_data)) + ' done. It was ' + eq_name)
    count += 1

earthquake number 0 of 3386 done. It was 20190131_194949.a
earthquake number 1 of 3386 done. It was 20190131_130953.a
earthquake number 2 of 3386 done. It was 20190131_130350.a
earthquake number 3 of 3386 done. It was 20190131_101656.a
earthquake number 4 of 3386 done. It was 20190131_085520.a
earthquake number 5 of 3386 done. It was 20190131_082011.a
earthquake number 6 of 3386 done. It was 20190131_054201.a
earthquake number 7 of 3386 done. It was 20190131_044030.a
earthquake number 8 of 3386 done. It was 20190131_035250.a
earthquake number 9 of 3386 done. It was 20190131_031714.a
earthquake number 10 of 3386 done. It was 20190131_031336.a
earthquake number 11 of 3386 done. It was 20190131_001648.a
earthquake number 12 of 3386 done. It was 20190130_231259.a
earthquake number 13 of 3386 done. It was 20190130_225434.a
earthquake number 14 of 3386 done. It was 20190130_221651.a
earthquake number 15 of 3386 done. It was 20190130_210253.a
earthquake number 16 of 3386 done. It was 20190130

/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))


earthquake number 76 of 3386 done. It was 20190124_141219.a
earthquake number 77 of 3386 done. It was 20190124_131330.a
earthquake number 78 of 3386 done. It was 20190124_123034.a
earthquake number 79 of 3386 done. It was 20190124_121517.a
earthquake number 80 of 3386 done. It was 20190124_112024.a
earthquake number 81 of 3386 done. It was 20190124_102829.a
earthquake number 82 of 3386 done. It was 20190124_080610.a
earthquake number 83 of 3386 done. It was 20190124_030457.a
earthquake number 84 of 3386 done. It was 20190124_001614.a
earthquake number 85 of 3386 done. It was 20190123_213423.a
earthquake number 86 of 3386 done. It was 20190123_132806.a
earthquake number 87 of 3386 done. It was 20190123_130902.a
earthquake number 88 of 3386 done. It was 20190123_112256.a
earthquake number 89 of 3386 done. It was 20190123_100900.a
earthquake number 90 of 3386 done. It was 20190123_054148.a
earthquake number 91 of 3386 done. It was 20190123_040056.a
earthquake number 92 of 3386 done. It wa

/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: CZ_MORC__HHE_D: Warning: Number of blockettes in fixed header (1) does not match the number parsed (2)
  warnings.warn(_w, InternalMSEEDWarning)
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: CZ_MORC__HHN_D: Warning: Number of blockettes in fixed header (1) does not match the number parsed (2)
  warnings.warn(_w, InternalMSEEDWarning)
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: CZ_MORC__HHZ_D: Warning: Number of blockettes in fixed header (1) does not match the number parsed (2)
  warnings.warn(_w, InternalMSEEDWarning)
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/mseed/headers.py:828: InternalMSEEDWarning: GE_MORC__HHE_D: Warning: Number of blockettes in fixed header (1) does not match the number parsed (2

earthquake number 94 of 3386 done. It was 20190122_223529.a
earthquake number 95 of 3386 done. It was 20190122_213050.a
earthquake number 96 of 3386 done. It was 20190122_201206.a
earthquake number 97 of 3386 done. It was 20190122_161926.a
earthquake number 98 of 3386 done. It was 20190122_152200.a
earthquake number 99 of 3386 done. It was 20190122_150049.a
earthquake number 100 of 3386 done. It was 20190122_134447.a
earthquake number 101 of 3386 done. It was 20190122_103750.a
earthquake number 102 of 3386 done. It was 20190122_082446.a
earthquake number 103 of 3386 done. It was 20190122_082444.a
earthquake number 104 of 3386 done. It was 20190122_045611.a
earthquake number 105 of 3386 done. It was 20190122_044319.a
earthquake number 106 of 3386 done. It was 20190122_043311.a
earthquake number 107 of 3386 done. It was 20190122_031928.a
earthquake number 108 of 3386 done. It was 20190122_022514.a
earthquake number 109 of 3386 done. It was 20190122_015034.a
earthquake number 110 of 3386 

...

earthquake number 2801 of 3352 done. It was 20190716_131736.a
earthquake number 2803 of 3351 done. It was 20190716_131450.a
earthquake number 2804 of 3351 done. It was 20190716_111230.a
earthquake number 2806 of 3350 done. It was 20190716_104537.a
earthquake number 2807 of 3350 done. It was 20190716_052652.a
earthquake number 2809 of 3349 done. It was 20190716_033211.a
earthquake number 2810 of 3349 done. It was 20190716_024705.a
earthquake number 2811 of 3349 done. It was 20190716_023552.a
earthquake number 2813 of 3348 done. It was 20190715_213512.a
earthquake number 2816 of 3346 done. It was 20190715_204650.a
earthquake number 2817 of 3346 done. It was 20190715_194349.a
earthquake number 2819 of 3345 done. It was 20190715_165819.a
earthquake number 2820 of 3345 done. It was 20190715_154336.a
earthquake number 2821 of 3345 done. It was 20190715_122544.a
earthquake number 2822 of 3345 done. It was 20190715_115405.a
earthquake number 2823 of 3345 done. It was 20190715_085053.a
earthqua

/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/

earthquake number 2853 of 3337 done. It was 20190714_084422.a
earthquake number 2854 of 3337 done. It was 20190714_072610.a
earthquake number 2855 of 3337 done. It was 20190714_071219.a
earthquake number 2856 of 3337 done. It was 20190714_063745.a
earthquake number 2860 of 3334 done. It was 20190714_032411.a
earthquake number 2861 of 3334 done. It was 20190714_022509.a
earthquake number 2862 of 3334 done. It was 20190714_022310.a
earthquake number 2863 of 3334 done. It was 20190714_015716.a
earthquake number 2864 of 3334 done. It was 20190714_015245.a
earthquake number 2867 of 3332 done. It was 20190713_222542.a
earthquake number 2868 of 3332 done. It was 20190713_222152.a
earthquake number 2870 of 3331 done. It was 20190713_214033.a
earthquake number 2871 of 3331 done. It was 20190713_211218.a
earthquake number 2873 of 3330 done. It was 20190713_175646.a
earthquake number 2874 of 3330 done. It was 20190713_173651.a
earthquake number 2875 of 3330 done. It was 20190713_155111.a
earthqua

/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))
/home/earthquakes1/software/Mageia/lib/python3.7/site-packages/obspy/io/stationxml/core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))


earthquake number 2981 of 3269 done. It was 20190710_194552.a
earthquake number 2990 of 3261 done. It was 20190710_145814.a
earthquake number 2998 of 3254 done. It was 20190710_121608.a


In [6]:
import pandas as pd

data = {
    'magnitude': list_mags,
    'tpmax': list_tpmax,
    'tc': list_tc
}

df = pd.DataFrame(data, index = list_eq)

df.to_pickle(root+'dataframe_highpass_0_075Hz.pkl')